# Administración del acceso a Amazon S3 con puntos finales de VPC y puntos de acceso a S3

Muchos clientes poseen varios buckets de Amazon S3, a algunos de los cuales acceden aplicaciones que se ejecutan en VPC. Amazon Virtual Private Cloud (Amazon VPC) le permite lanzar recursos de AWS en una red virtual que usted define. Esta red virtual se asemeja mucho a una red tradicional que operaría en su propio centro de datos, con los beneficios de utilizar la infraestructura escalable de AWS. A menudo se quiere asegurar que las aplicaciones que se ejecutan dentro de una VPC tengan acceso sólo a cubos S3 específicos.  Además, cuando tiene varios conjuntos de datos compartidos a los que deben acceder las aplicaciones que se ejecutan en diferentes VPC, la gestión del acceso y los permisos puede convertirse rápidamente en un reto.

Los puntos finales de la VPC para Amazon S3 simplifican el acceso a S3 desde una VPC proporcionando conexiones seguras configurables y altamente fiables a S3 que no requieren una puerta de enlace de Internet o un dispositivo de traducción de direcciones de red (NAT). Al crear un punto de enlace de la VPC de S3, puede adjuntarle una política de punto de enlace que controle el acceso a Amazon S3.

Los puntos de acceso de S3, una característica de Amazon S3, simplifican la administración del acceso a los datos a escala para las aplicaciones que utilizan conjuntos de datos compartidos en S3. Los puntos de acceso son nombres de host únicos que los clientes crean para aplicar distintos permisos y controles de red para cualquier solicitud realizada a través del punto de acceso.

En esta entrada, hablo de un enfoque que utiliza los puntos de acceso de S3 en combinación con las políticas de punto final de la VPC para facilitar la administración del acceso a los conjuntos de datos compartidos en Amazon S3. La idea es crear un punto de acceso exclusivo de la VPC de Amazon S3 y, a continuación, utilizarlo en la política de punto final de la VPC para controlar el acceso al bucket de S3. También tienes la opción de utilizar políticas de cubos para el acceso a los cubos de S3 a través de cortafuegos sólo a las VPC, que también cubro.

### Uso de los puntos finales de la VPC de Amazon S3 para controlar el acceso a los buckets de S3

Las organizaciones pueden especificar cubos individuales en una política de puntos finales de la VPC de Amazon S3, lo que les permite garantizar que solo se puede acceder a cubos específicos desde su VPC (es decir, cuando se está dentro de la VPC, solo se puede acceder a determinados cubos).

A continuación se muestra un ejemplo de política de punto final de VPC para permitir el acceso a un bucket de S3 específico desde dentro de una VPC:

In [ ]:
{
  "Statement": [
    {
      "Sid": "Access-to-specific-bucket-only",
      "Principal": "*",
      "Action": [
        "s3:GetObject",
        "s3:PutObject"
      ],
      "Effect": "Allow",
      "Resource": ["arn:aws:s3:::my_secure_bucket",
                   "arn:aws:s3:::my_secure_bucket/*"]
    }
  ]
}

Si bien esto es útil, a medida que el número de buckets de la organización crece, se vuelve difícil hacer un seguimiento y especificar manualmente los buckets recién creados en la política de endpoints de la VPC de Amazon S3. Por ejemplo, cuando se crea un nuevo bucket de S3 en una cuenta concreta a la que la aplicación que se ejecuta dentro de una VPC necesita acceder, hay que editar manualmente la política de endpoints de la VPC para permitir la lista del bucket de S3 recién creado.

Para simplificar la gestión de esto, nos fijamos en los puntos de acceso de Amazon S3.

### Puntos de Acceso S3
Los Puntos de Acceso S3 son nombres de host únicos que se pueden crear para imponer permisos y controles de red distintos para cualquier solicitud realizada a través del Punto de Acceso.

Algunas características clave de los Puntos de Acceso S3:

* Los Puntos de Acceso contienen un nombre de host, un ARN de AWS y una política de recursos de AWS IAM.
* Los Puntos de Acceso tienen por defecto una configuración específica para Bloquear el Acceso Público.
* Los puntos de acceso son únicos para una cuenta y una región.
* Los Puntos de Acceso pueden tener permisos IAM personalizados para un usuario o aplicación.
* Los Puntos de Acceso pueden tener permisos IAM personalizados para objetos específicos en un cubo a través de un prefijo para controlar con precisión el acceso.
* Los puntos de acceso pueden configurarse para aceptar solicitudes solo de una nube privada virtual (VPC) para restringir el acceso a los datos de Amazon S3 a una red privada.

La siguiente imagen muestra un ejemplo de cómo puede utilizar los puntos de acceso de S3 para administrar el acceso a conjuntos de datos compartidos en Amazon S3.
<br>
<br>
<img src="https://d2908q01vomqb2.cloudfront.net/e1822db470e60d090affd0956d743cb0e7cdf113/2020/10/12/One-example-of-how-you-can-use-S3-Access-Points-to-manage-access-to-shared-datasets-on-Amazon-S3..png" width="900">

**Nota**: Antes de utilizar endpoints con Amazon S3, asegúrese de haber leído las siguientes limitaciones generales: Limitaciones de los endpoints de la puerta de enlace.



### Uso de puntos de acceso de S3 con endpoints de VPC

Para restringir aún más el acceso a un bucket compartido de Amazon S3, puede utilizar una política de endpoints de VPC para exigir que las aplicaciones utilicen el punto de acceso de S3 a través de una VPC específica.

Los puntos de acceso de S3 tienen un ARN de AWS que incluye el número de cuenta y el identificador de región, que puede utilizarse en la política de endpoints de la VPC. En lugar de especificar buckets individuales en la política de endpoints de la VPC de Amazon S3, se puede utilizar un prefijo de Access Point para especificar todos los Access Points de una cuenta.

Por ejemplo, en la política de endpoints de la VPC, puede añadir una condición como se muestra en el siguiente fragmento:

In [ ]:
"Condition": {
                "ArnNotLikeIfExists": {
                    "s3:DataAccessPointArn": "arn:aws:s3:us-east-1:<Account ID>:accesspoint/*"
                }

Cuando se crea un nuevo bucket de Amazon S3, para permitir el acceso desde la VPC, puede crear un punto de acceso de S3 en el bucket de S3. La condición anterior en la política de endpoints de la VPC permitiría automáticamente el acceso a este nuevo bucket de S3 a través del Access Point, sin tener que editar la política de endpoints de la VPC.

### Configuración y tutorial

A continuación veremos cómo configurar los S3 Access Points para un bucket de Amazon S3 y utilizarlo con los endpoints de la VPC. El siguiente diagrama muestra la configuración en su totalidad:
<br>
<br>
<img src="https://d2908q01vomqb2.cloudfront.net/e1822db470e60d090affd0956d743cb0e7cdf113/2020/10/12/Set-up-S3-Access-Points-for-an-Amazon-S3-bucket-and-use-it-with-VPC-endpoints.png" width="900">

In [1]:
import boto3
import json
import random

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [2]:
# create VPC
vpc = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [3]:
# Crear routable
routetable = vpc.create_route_table()
routetable.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-Private-RT"}])

[ec2.Tag(resource_id='rtb-0d890bbdab9b08f26', key='Name', value='AWSCookbookVPC-Private-RT')]

In [4]:
# Cree una subrede:
subnet = ec2.create_subnet(
    CidrBlock='10.10.1.0/24', 
    VpcId=vpc.id,
    AvailabilityZone='us-east-1a'
)
subnet.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-Private-SN"}])

[ec2.Tag(resource_id='subnet-0d97e8754c87150d3', key='Name', value='AWSCookbook-Private-SN')]

In [5]:
# Asociar la tabla de rutas con la subred:
routetable.associate_with_subnet(SubnetId = subnet.id)

ec2.RouteTableAssociation(id='rtbassoc-0b57ebbe6bbbe6da6')

In [6]:
# Crear dos bucket
bucket_name_1 = 'reportbucket-{}-allowed'.format(random.randint(1000, 100000))
bucket_name_2 = 'reportbucket-{}-not-allowed'.format(random.randint(1000, 100000))
bucket_1 = s3.create_bucket(Bucket=bucket_name_1)
bucket_2 = s3.create_bucket(Bucket=bucket_name_2)
print("Bucket (1) name: {}".format(bucket_name_1))
print("Bucket (2) name: {}".format(bucket_name_2))

Bucket (1) name: reportbucket-75007-allowed
Bucket (2) name: reportbucket-96727-not-allowed


In [7]:
response_1 = s3_client.put_public_access_block(
    Bucket=bucket_name_1,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    }
)
response_2 = s3_client.put_public_access_block(
    Bucket=bucket_name_2,
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    }
)

A continuación crearemos un punto de acceso solo para la VPC para el bucket de Amazon S3. Esto garantiza que solo puedan acceder a este punto de acceso los recursos de una VPC específica.

### Añadir el punto de acceso de la VPC de Amazon S3 sólo en los cubos

En primer lugar, creamos un punto de acceso de S3 al que solo se puede acceder desde una VPC específica. Tiene un origen de red de VPC, y Amazon S3 rechaza cualquier solicitud realizada al punto de acceso que no se origine en esa VPC.

In [20]:
s3control = boto3.client('s3control')

In [21]:
# Get Account ID
account_id = boto3.client('sts').get_caller_identity().get('Account')

In [27]:
response = s3control.create_access_point(
    Bucket=bucket_name_1,
    AccountId=account_id,
    Name='vpc-only-access-point',
    VpcConfiguration={
        'VpcId': vpc.id
    },
    PublicAccessBlockConfiguration={
        'BlockPublicAcls': True,
        'IgnorePublicAcls': True,
        'BlockPublicPolicy': True,
        'RestrictPublicBuckets': True
    }
)

In [29]:
arn_access_point = response['AccessPointArn']

In [37]:
policy_endpoint = {
    "Version": "2008-10-17",
    "Statement": [
        {
            "Sid": "AllowUseOfS3",
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:*",
            "Resource": "*"
        },
        {
            "Sid": "OnlyIfAccessedViaAccessPoints",
            "Effect": "Deny",
            "Principal": "*",
            "Action": "s3:*",
            "Resource": "*",
            "Condition": {
                "ArnNotLikeIfExists": {
                    "s3:DataAccessPointArn": arn_access_point
                }
            }
        }
    ]
}

In [38]:
# Create an Amazon S3 gateway endpoint in your VPC
gateway_endpoint = ec2_client.create_vpc_endpoint(
    VpcId=vpc.id,
    ServiceName='com.amazonaws.{}.s3'.format(region_aws),
    VpcEndpointType='Gateway',
    RouteTableIds=[
        routetable.id,
    ],
    PolicyDocument=json.dumps(policy_endpoint)
)

In [40]:
# Add tags to the endpoint
response = ec2_client.create_tags(
    Resources=[
        gateway_endpoint['VpcEndpoint']['VpcEndpointId'],
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'S3GatewayEndpoint'
        },
    ]
)

### Bloquear el bucket de Amazon S3 sólo para el acceso a la VPC

Es posible que desee asegurarse de que solo se pueda acceder a los buckets de Amazon S3 desde una VPC. Esto puede lograrse utilizando una política de bucket en el bucket de S3 que restrinja el acceso solo a las VPC especificadas.

In [42]:
# create an S3 bucket policy:
bucket_policy = {
    "Version": "2012-10-17",
    "Id": "S3BukcetPolicyVPCAccessOnly",
    "Statement": [
        {
            "Sid": "DenyIfNotFromAllowedVPC",
            "Effect": "Deny",
            "Principal": "*",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name_1),
                "arn:aws:s3:::{}/*".format(bucket_name_1)
            ],
            "Condition": {
                "StringNotEquals": {
                    "aws:SourceVpc": vpc.id
                }
            }
        }
    ]
}

In [44]:
# Attach the policy to the bucket:
bucket_policy = json.dumps(bucket_policy)
response = s3.BucketPolicy(bucket_name_1).put(Policy=bucket_policy)

{'ResponseMetadata': {'RequestId': '2FS4CDTX5Q3QVPH8',
  'HostId': 'HpELKGCJXh/kKt5UpdYkuHLaveu8GI09C+9YTifKQ3nXUUS4u6VqtMlRthxl5qWbiwx3L9+0vY4=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'HpELKGCJXh/kKt5UpdYkuHLaveu8GI09C+9YTifKQ3nXUUS4u6VqtMlRthxl5qWbiwx3L9+0vY4=',
   'x-amz-request-id': '2FS4CDTX5Q3QVPH8',
   'date': 'Fri, 14 Oct 2022 00:59:17 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Conclusión

Administrar el acceso a los datos a escala para los conjuntos de datos compartidos en Amazon S3 puede ser un reto cuando se tiene un gran número de aplicaciones con diferentes requisitos de acceso. Además, garantizar que el acceso a los datos confidenciales esté protegido por un firewall dentro de sus redes privadas puede aumentar el desafío.

En este artículo, he explicado cómo puede utilizar los endpoints de la VPC de Amazon S3 y los puntos de acceso de S3 para administrar los permisos de los conjuntos de datos compartidos en S3. También hablé de cómo se puede hacer un firewall de acceso a los datos dentro de su VPC para asegurar que sus datos sensibles están protegidos de cualquier acceso no intencionado desde fuera de sus VPCs.

Los S3 Access Points se pueden utilizar con endpoints de VPC para proporcionar un acceso seguro a buckets de S3 multi-tenant al tiempo que facilitan la gestión de permisos. Disponer de un acceso seguro a los buckets de S3 de varios inquilinos al tiempo que se gestionan fácilmente los permisos le permite escalar sin problemas con una intervención manual mínima, al tiempo que garantiza la protección de sus datos confidenciales.